In [1]:
import os
import pandas as pd
import numpy as np
import datetime as date
import datetime as dt

DATA_ROOT = "./data/ml100marathon-02-01"

In [2]:
#Import Data
off_train = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_train = off_train[~off_train.coupon_id.isna()]
off_train.reset_index(drop=True, inplace=True)

off_test = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
off_test.columns  = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']


In [3]:
off_train.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
1,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
2,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
3,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
4,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN


In [4]:
off_train.dtypes

user_id            int64
merchant_id        int64
coupon_id        float64
discount_rate     object
distance         float64
date_received    float64
date             float64
dtype: object

In [5]:
# 按照時間劃分訓練集和測試集
# siding windows
# 本賽題提供用戶在2016年1月1日至2016年5月31日之間真實線下消費行為

#2016-1-1~2016-2-29 trian set(feature extract) , 2016-3-1~2016-3-31 test set
#2016-2-1~2016-3-31 train set(feature extract) , 2016-4-1~2016-4-30 test set
#2016-3-1~2016-4-30 train set(feature extract) , 2016-5-1~2016-5-31 test set

#dataset=>test set，feature=>train set


#數據集3 線上數據中領券和用券日期大於3月1日和小於4月30日的
#将3月15日到4月30日作为数据集提取特征，再测试5月1日到6月15日的数据
#使数据集3等于test集

#dataset裡面只有接收優惠券的記錄的，無消費記錄，可用於預測
#feature裡面存的是優惠券使用日期或接收優惠券的時間介於3月15日到6月30之間的記錄

dataset3 = off_test
feature3 = off_train[(off_train.date>=20160301)&(off_train.date<=20160430)|((pd.isnull(off_train.date)&(off_train.date_received>=20160301)&(off_train.date_received<=20160430)))]
 
#提取數據集2的測試集
#將2月1日到3月31日的作為數據集提取特徵，利用4月1日4月30日的作為測試集

#dataset裡面只存放優惠券接收日期介於4月1日到4月30之間的記錄
#feature裡面存的是優惠券使用日期或接收優惠券的時間介於2月1日到3月31之間的記錄
dataset2 = off_train[((off_train.date_received>=20160401)&(off_train.date_received<=20160430))]
feature2 = off_train[(off_train.date>=20160201)&(off_train.date<=20160331)|(pd.isnull(off_train.date)&(off_train.date_received>=20160201)&(off_train.date_received<=20160331))]

#提取數據集1的測試集
#dataset裡面只存放優惠券接收日期介於3月1日到3月31之間的記錄
#feature裡面存的是優惠券使用日期或接收優惠券的時間介於1月1日到2月29之間的記錄
dataset1 = off_train[(off_train.date_received>=20160301)&(off_train.date_received<=20160331)]
feature1 = off_train[(off_train.date>=20160101)&(off_train.date<=20160229)|(pd.isnull(off_train.date)&(off_train.date_received>=20160101)&(off_train.date_received<=20160229))]

In [6]:
# 提取特徵：
    #用戶領取的所有優惠券數目
    #用戶領取的特定優惠券數目
    #用戶此次之後/前領取的所有優惠券數目
    #用戶此次之後/前領取的特定優惠券數目
    #用戶上/下一次領取的時間間隔
    #用戶領取特定商家的優惠券數目
    #用戶領取的不同商家數目
    #用戶當天領取的優惠券數目
    #用戶當天領取的特定優惠券數目
    #用戶領取的所有優惠券種類數目
    #商家被領取的優惠券數目
    #商家被領取的特定優惠券數目
    #商家被多少不同用戶領取的數目
    #商家發行的所有優惠券種類數目

# 對dataset3進行操作
# 用戶收到的優惠券總和
t = dataset3[['user_id']]
t['this_month_user_received_all_coupon_count'] = 1
 #將t按照用戶id進行分組，然後統計所有用戶收取的優惠券數目,並初始化一個索引值
t = t.groupby('user_id').agg('sum').reset_index()
# 用戶收到特定優惠券的總和
t1 = dataset3[['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
 
# 用戶此次之前或之後領使用優惠券的時間
# lambda x:':'.join(x)  是添加冒號並在後面去加字符
# 將接收時間的一組按著':'分開，這樣就可以計算接受了優惠券的數量,apply是合併
# 最大接受的日期max_date_received/min_date_received
t2 = dataset3[['user_id','coupon_id','date_received']]
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(lambda s:min([float(d) for d in s.split(':')]))
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
 
# 將表格中接收優惠券日期中為最近和最遠的日期時置為1其餘為0，若只接受了一次優惠券為-1
# 將兩表融合只保留左表數據,這樣得到的表，相當於保留了最近接收時間和最遠接受時間
t3 = dataset3[['user_id','coupon_id','date_received']]
t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received.astype(float)
 
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received.astype(float) - t3.min_date_received
 
def isfirstlastone(x):
     
    if x == 0:
        return 1
    elif x > 0:
        return 0
    else:
        return -1
# 只接受過一次優惠券為者為 -1
 
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(isfirstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(isfirstlastone)
 
# 第四個特徵,一個用戶所接收到的所有優惠券的數量
t4 = dataset3[['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
 
# 提取第五個特徵,一個用戶不同時間所接收到不同優惠券的數量
t5 = dataset3[['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
 
# 一個用戶不同優惠券 的接受時間
t6 = dataset3[['user_id','coupon_id','date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns ={'date_received':'dates'},inplace = True)
 
# 接收優惠券最近的日子天數
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    
    gaps = []
    for d in dates:
        if len(d) > 4:
            this_gap = (dt.date(int(date_received[1:4]),(int(date_received[4:6])),(int(date_received[6:8]))) - dt.date((int(d[1:4])),(int(d[4:6])),(int(d[6:8])))).days
            if this_gap>0:
                gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)
# 接收優惠券最遠的日子天數
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        if len(d) > 4:
            this_gap = (dt.datetime(int(d[0:4]),int(d[4:6]),int(d[6:8])) - dt.datetime(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)
 
t7 = dataset3[['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates.astype('str')
 
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after']  = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]
# feature3 提取的特徵存入CSV中
other_feature3 = pd.merge(t1,t,on='user_id')
other_feature3 = pd.merge(other_feature3,t3,on=['user_id','coupon_id'])
other_feature3 = pd.merge(other_feature3,t4,on=['user_id','date_received'])
other_feature3 = pd.merge(other_feature3,t5,on=['user_id','coupon_id','date_received'])
other_feature3 = pd.merge(other_feature3,t7,on=['user_id','coupon_id','date_received'])
other_feature3.to_csv('data/ml100marathon-02-01/other_feature3.csv',index=None)
 
 
 
# 處理dataset2
t = dataset2[['user_id']]
t['this_month_user_received_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()
t1 = dataset2[['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
t2 = dataset3[['user_id','coupon_id','date_received']]
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
 
t3 = dataset2[['user_id','coupon_id','date_received']]
t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received.astype('float')
t3['this_month_user_receive_same_coupon_firstone']= t3.date_received.astype('float') - t3.min_date_received
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(isfirstlastone)
t3.this_month_user_receive_same_coupon_firstone= t3.this_month_user_receive_same_coupon_firstone.apply(isfirstlastone)
 
t4 = dataset2[['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
t5 = dataset2[['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
 
t6 = dataset2[['user_id','coupon_id','date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)
 
t7 = dataset2[['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after']  = t7.date_received_date.apply(get_day_gap_before)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]
 
other_feature2 = pd.merge(t1,t,on='user_id')
other_feature2 = pd.merge(other_feature2,t3,on=['user_id','coupon_id'])
other_feature2 = pd.merge(other_feature2,t4,on=['user_id','date_received'])
other_feature2 = pd.merge(other_feature2,t5,on=['user_id','coupon_id','date_received'])
other_feature2 = pd.merge(other_feature2,t7,on=['user_id','coupon_id','date_received'])
other_feature2.to_csv('data/ml100marathon-02-01/other_feature2.csv',index=None)
 
# 處理dataset1
t = dataset1[['user_id']]
t['this_month_user_received_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()
t1 = dataset1[['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
t2 = dataset1[['user_id','coupon_id','date_received']]
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
 
t3 = dataset1[['user_id','coupon_id','date_received']]
t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received.astype('float')
t3['this_month_user_receive_same_coupon_firstone']= t3.date_received.astype('float') - t3.min_date_received
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(isfirstlastone)
t3.this_month_user_receive_same_coupon_firstone= t3.this_month_user_receive_same_coupon_firstone.apply(isfirstlastone)
 
t4 = dataset1[['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
t5 = dataset1[['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
 
t6 = dataset1[['user_id','coupon_id','date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)
 
t7 = dataset1[['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after']  = t7.date_received_date.apply(get_day_gap_before)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]
 
other_feature1 = pd.merge(t1,t,on='user_id')
other_feature1 = pd.merge(other_feature1,t3,on=['user_id','coupon_id'])
other_feature1 = pd.merge(other_feature1,t4,on=['user_id','date_received'])
other_feature1 = pd.merge(other_feature1,t5,on=['user_id','coupon_id','date_received'])
other_feature1 = pd.merge(other_feature1,t7,on=['user_id','coupon_id','date_received'])

other_feature1.to_csv('data/ml100marathon-02-01/other_feature1.csv',index=None)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

In [7]:
import datetime

#提取優惠券相關特徵
# 統一轉化為打折卷
def calc_discount_rate(s):
    s = str(s)
    s = s.split(':')
    if len(s) == 1:
        return float(s[0])
    else:
        return 1.0-float(s[1])/float(s[0])
def get_discount_man(s):
    s = str(s)
    s = s.split(':')
    if len(s) == 1:
        return 'null'
    else:
        return int(s[0])
def get_discount_jian(s):
    s = str(s)
    s = s.split(':')
    if len(s) == 1:
        return 'null'
    else:
        return int(s[1])
def is_man_jian(s):
    s = str(s)
    s = s.split(':')
    if len(s) == 1:
        return 0
    else:
        return 1
    
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return -1
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1

def getMonth(row):
    if (np.isnan(row)) or (row==-1):
        return -1
    else:
        x = pd.to_datetime(row, format = "%Y%m%d")
        return datetime.datetime.strftime(x, '%m')

def getDaysDistance1(row):
    if (np.isnan(row)) or (row==-1):
        return -1
    else:
        d1 = pd.to_datetime(row, format = "%Y%m%d").date()
        d2 = dt.date(2016,4,30)
        delta = d1 - d2
        return delta.days
    
def getDaysDistance2(row):
    if (np.isnan(row)) or (row==-1):
        return -1
    else:
        d1 = pd.to_datetime(row, format = "%Y%m%d").date()
        d2 = dt.date(2016,3,31)
        delta = d1 - d2
        return delta.days
    
def getDaysDistance3(row):
    if (np.isnan(row)) or (row==-1):
        return -1
    else:
        d1 = pd.to_datetime(row, format = "%Y%m%d").date()
        d2 = dt.date(2016,2,29)
        delta = d1 - d2
        return delta.days
    
# 處理數據集3，處理時間屬性，顯示時間是第幾周
dataset3['day_of_week']  = dataset3.date_received.apply(getWeekday)
dataset3['day_of_month'] = dataset3.date_received.apply(getMonth).astype('int64')
dataset3['days_distance']= dataset3.date_received.apply(getDaysDistance1).astype('int64')
dataset3['discount_man'] = dataset3.discount_rate.apply(get_discount_man)
dataset3['discount_jian']= dataset3.discount_rate.apply(get_discount_jian)
dataset3['is_man_jian']  = dataset3.discount_rate.apply(is_man_jian)
dataset3['discount_rate']= dataset3.discount_rate.apply(calc_discount_rate)
d = dataset3[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset3 = pd.merge(dataset3,d,on='coupon_id',how='left')
dataset3.to_csv('data/ml100marathon-02-01/coupon3_feature.csv',index=None)
 
# 數據集2
dataset2['day_of_week']  = dataset2.date_received.apply(getWeekday)
dataset2['day_of_month'] = dataset2.date_received.apply(getMonth).astype('int64')
dataset2['days_distance']= dataset2.date_received.apply(getDaysDistance2).astype('int64')
dataset2['discount_man'] = dataset2.discount_rate.apply(get_discount_man)
dataset2['discount_jian']= dataset2.discount_rate.apply(get_discount_jian)
dataset2['is_man_jian']  = dataset2.discount_rate.apply(is_man_jian)
dataset2['discount_rate']= dataset2.discount_rate.apply(calc_discount_rate)
d = dataset2[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset2 = pd.merge(dataset2,d,on='coupon_id',how='left')
dataset2.to_csv('data/ml100marathon-02-01/coupon2_feature.csv',index=None)
 
# 數據集1
dataset1['day_of_week']  = dataset1.date_received.apply(getWeekday)
dataset1['day_of_month'] = dataset1.date_received.apply(getMonth).astype('int64')
dataset1['days_distance']= dataset1.date_received.apply(getDaysDistance3).astype('int64')
dataset1['discount_man'] = dataset1.discount_rate.apply(get_discount_man)
dataset1['discount_jian']= dataset1.discount_rate.apply(get_discount_jian)
dataset1['is_man_jian']  = dataset1.discount_rate.apply(is_man_jian)
dataset1['discount_rate']= dataset1.discount_rate.apply(calc_discount_rate)
d = dataset1[['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset1 = pd.merge(dataset1,d,on='coupon_id',how='left')
dataset1.to_csv('data/ml100marathon-02-01/coupon1_feature.csv',index=None)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [8]:
#提取商戶相關特徵
merchant3 = feature3[['merchant_id','coupon_id','distance','date_received','date']]
t = merchant3[['merchant_id']]
# 刪除重復的行數據
t.drop_duplicates(inplace=True)
# 顯示賣出的商品,以及賣出的數量
# []用來強調條件\或者新建一列並賦值   [[]]用來表示取哪個列來使用
t1 = merchant3[pd.notnull(merchant3.date)][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
# 顯示使用了優惠券消費的商品，正樣本
t2 = merchant3[(pd.notnull(merchant3.date))&(pd.notnull(merchant3.coupon_id))][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()
# 提取商品優惠券的總數量
t3 = merchant3[pd.notnull(merchant3.coupon_id)][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()
# 提取銷量與距離的關係
# 把數據中的空值全部替換為 -1
t4 = merchant3[(pd.notnull(merchant3.date))&(pd.notnull(merchant3.coupon_id))][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.distance = t4.distance.astype('float')
t4.replace(-1,np.nan,inplace=True)
# 提取用戶和商店距離的最小值
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace = True)
# 提取用戶和商店距離的最大值
t6 = t4.groupby('merchant_id').agg('max').reset_index()
t5.rename(columns={'distance':'merchant_max_distance'},inplace = True)
# 提取用戶和商品距離的平均值
t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace = True)
# 提取用戶與商品距離的中位數
t8 = t4.groupby('merchant_id').agg('median').reset_index()
# 把特徵集合入一張表裡
merchant3_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t2,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t3,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t5,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t6,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t7,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t8,on='merchant_id',how='left')
# merchant3_feature.head()
# 替換數據中的NAN為0，便於計算優惠券的使用率以及其他信息
# 優惠券的使用率、賣出的商品中使用優惠券的佔比
merchant3_feature.sales_use_coupon = merchant3_feature.sales_use_coupon.replace(np.nan,0)
merchant3_feature['merchant_coupon_transfer_rate'] = merchant3_feature.sales_use_coupon.astype('float') / merchant3_feature.total_sales
merchant3_feature['coupon_rate'] = merchant3_feature.sales_use_coupon.astype('float') / merchant3_feature.total_sales
merchant3_feature.total_coupon = merchant3_feature.total_coupon.replace(np.nan,0)
merchant3_feature.to_csv('data/ml100marathon-02-01/merchant3_feature.csv',index=None)
 
 
#對feature2進行操作
merchant2 = feature2[['merchant_id','coupon_id','distance','date_received','date']]
t = merchant2[['merchant_id']]
# 刪除重復的行數據
t.drop_duplicates(inplace=True)
# 顯示賣出的商品,以及賣出的數量
# []用來強調條件\或者新建一列並賦值   [[]]用來表示取哪個列來使用
t1 = merchant2[pd.notnull(merchant2.date)][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
# 顯示使用了優惠券消費的商品，正樣本
t2 = merchant2[(pd.notnull(merchant2.date))&(pd.notnull(merchant2.coupon_id))][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()
# 提取商品優惠券的總數量
t3 = merchant2[pd.notnull(merchant2.coupon_id)][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()
# 提取銷量與距離的關係
# 把數據中的空值全部替換為 -1
t4 = merchant2[(pd.notnull(merchant2.date))&(pd.notnull(merchant2.coupon_id))][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.distance = t4.distance.astype('float')
t4.replace(-1,np.nan,inplace=True)
# 提取用戶和商店距離的最小值
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace = True)
# 提取用戶和商店距離的最大值
t6 = t4.groupby('merchant_id').agg('max').reset_index()
t5.rename(columns={'distance':'merchant_max_distance'},inplace = True)
# 提取用戶和商品距離的平均值
t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace = True)
# 提取用戶與商品距離的中位數
t8 = t4.groupby('merchant_id').agg('median').reset_index()
# 把特徵集合入一張表裡
merchant2_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t2,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t3,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t5,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t6,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t7,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t8,on='merchant_id',how='left')
# merchant3_feature.head()
# 替換數據中的NAN為0，便於計算優惠券的使用率以及其他信息
# 優惠券的使用率、賣出的商品中使用優惠券的佔比
merchant2_feature.sales_use_coupon = merchant2_feature.sales_use_coupon.replace(np.nan,0)
merchant2_feature['merchant_coupon_transfer_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_sales
merchant2_feature['coupon_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_sales
merchant2_feature.total_coupon = merchant2_feature.total_coupon.replace(np.nan,0)
merchant2_feature.to_csv('data/ml100marathon-02-01/merchant2_feature.csv',index=None)
 
 
 
# 對feature1進行操作
merchant1 = feature1[['merchant_id','coupon_id','distance','date_received','date']]
t = merchant1[['merchant_id']]
# 刪除重復的行數據
t.drop_duplicates(inplace=True)
# 顯示賣出的商品,以及賣出的數量
# []用來強調條件\或者新建一列並賦值   [[]]用來表示取哪個列來使用
t1 = merchant1[pd.notnull(merchant1.date)][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
# 顯示使用了優惠券消費的商品，正樣本
t2 = merchant1[(pd.notnull(merchant1.date))&(pd.notnull(merchant1.coupon_id))][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()
# 提取商品優惠券的總數量
t3 = merchant1[pd.notnull(merchant1.coupon_id)][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()
# 提取銷量與距離的關係
# 把數據中的空值全部替換為 -1
t4 = merchant1[(pd.notnull(merchant1.date))&(pd.notnull(merchant1.coupon_id))][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.distance = t4.distance.astype('float')
t4.replace(-1,np.nan,inplace=True)
# 提取用戶和商店距離的最小值
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace = True)
# 提取用戶和商店距離的最大值
t6 = t4.groupby('merchant_id').agg('max').reset_index()
t5.rename(columns={'distance':'merchant_max_distance'},inplace = True)
# 提取用戶和商品距離的平均值
t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace = True)
# 提取用戶與商品距離的中位數
t8 = t4.groupby('merchant_id').agg('median').reset_index()
# 把特徵集合入一張表裡
merchant1_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t2,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t3,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t5,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t6,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t7,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t8,on='merchant_id',how='left')
# merchant3_feature.head()
# 替換數據中的NAN為0，便於計算優惠券的使用率以及其他信息
# 優惠券的使用率、賣出的商品中使用優惠券的佔比
merchant1_feature.sales_use_coupon = merchant1_feature.sales_use_coupon.replace(np.nan,0)
merchant1_feature['merchant_coupon_transfer_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_sales
merchant1_feature['coupon_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_sales
merchant1_feature.total_coupon = merchant1_feature.total_coupon.replace(np.nan,0)
merchant1_feature.to_csv('data/ml100marathon-02-01/merchant1_feature.csv',index=None)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
#提取用戶的相關特徵
#用戶的相關信息：
    #count_merchant
    #user_avg_distance,user_min_distance,user_max_distance
    #buy_use_coupon,buy_total,coupon_received
    #buy_use_coupon/coupon_received
    #buy_use_coupon/buy_total
    #user_date_datereceived_gap

def get_user_date_datereceived_gap(s):
    s = s.split(':')
    if s[0]=='null':
        return 0
    if s[0]=='nan' :
        return 0
    if s[1]=='null':
        return 0
    if s[1]=='nan' :
        return 0
    else:
        return 0
        #return(dt.date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8])) - dt.date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days


# 數據集3的處理
user3 = feature3[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
t = user3[['user_id']]
# 去掉數據中重復的用戶ID
t.drop_duplicates(inplace=True)
# 用戶購買商品的種類
t1 = user3[pd.notnull(user3.date)][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)
# 使用了優惠券購買商品的用戶id和距離
t2 = user3[(pd.notnull(user3.date))&(pd.notnull(user3.coupon_id))][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.distance = t2.distance.astype('float')
t2.replace(-1,np.nan,inplace=True)
# 得到使用優惠券購買商品的用戶距商店的最短距離
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_dsitance'},inplace=True)
# 最大距離
t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)
# 平均距離
t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)
# 中位數距離
t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)
# 每個用戶使用優惠券購買的商品數量
t7 = user3[(pd.notnull(user3.date))&(pd.notnull(user3.coupon_id))][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()
# 購買商品的總數
t8 = user3[pd.notnull(user3.date)][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()
# 接收優惠券的總數
t9 = user3[pd.notnull(user3.coupon_id)][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()
# 收到優惠券的日期和使用之間的距離
t10 = user3[(pd.notnull(user3.date_received))&(pd.notnull(user3.date))][['user_id','date_received','date']]
t10['user_date_datereceived_gap'] = t10.date.to_string()+':'+t10.date_received.to_string()
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id','user_date_datereceived_gap']]
# 將用戶優惠券使用時間的間隔取平均值
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
# 間隔天數的最小值
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
# 間隔天數的最大值
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)
# 合併特徵
user3_feature = pd.merge(t,t1,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t3,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t4,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t5,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t6,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t7,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t8,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t9,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t11,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t12,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t13,on='user_id',how='left')
user3_feature.count_merchant = user3_feature.count_merchant.replace(np.nan,0)
user3_feature.buy_user_coupon = user3_feature.buy_use_coupon.replace(np.nan,0)
user3_feature['buy_use_coupon_rate'] = user3_feature.buy_use_coupon.astype('float')/user3_feature.buy_total.astype('float')
user3_feature['user_coupon_transfer_rate'] = user3_feature.buy_use_coupon.astype('float')/user3_feature.buy_use_coupon.astype('float')
user3_feature.buy_total = user3_feature.buy_total.replace(np.nan,0)
user3_feature.coupon_received = user3_feature.coupon_received.replace(np.nan,0)
user3_feature.to_csv('data/ml100marathon-02-01/user3_feature.csv',index=None)
 
 
# 對數據集faeture2操作
user2 = feature2[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
t = user2[['user_id']]
# 去掉數據中重復的用戶ID
t.drop_duplicates(inplace=True)
# 用戶購買商品的種類
t1 = user2[pd.notnull(user2.date)][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)
# 使用了優惠券購買商品的用戶id和距離
t2 = user2[(pd.notnull(user2.date))&(pd.notnull(user2.coupon_id))][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.distance = t2.distance.astype('float')
t2.replace(-1,np.nan,inplace=True)
# 得到使用優惠券購買商品的用戶距商店的最短距離
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_dsitance'},inplace=True)
# 最大距離
t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)
# 平均距離
t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)
# 中位數距離
t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)
# 每個用戶使用優惠券購買的商品數量
t7 = user2[(pd.notnull(user2.date))&(pd.notnull(user2.coupon_id))][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()
# 購買商品的總數
t8 = user2[pd.notnull(user2.date)][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()
# 接收優惠券的總數
t9 = user2[pd.notnull(user2.coupon_id)][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()
# 收到優惠券的日期和使用之間的距離
t10 = user2[(pd.notnull(user2.date_received))&(pd.notnull(user2.date))][['user_id','date_received','date']]
t10['user_date_datereceived_gap'] = t10.date.to_string()+':'+t10.date_received.to_string()
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id','user_date_datereceived_gap']]
# 將用戶優惠券使用時間的間隔取平均值
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
# 間隔天數的最小值
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
# 間隔天數的最大值
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)
# 合併特徵
user2_feature = pd.merge(t,t1,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t3,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t4,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t5,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t6,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t7,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t8,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t9,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t11,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t12,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t13,on='user_id',how='left')
user2_feature.count_merchant = user2_feature.count_merchant.replace(np.nan,0)
user2_feature.buy_user_coupon = user2_feature.buy_use_coupon.replace(np.nan,0)
user2_feature['buy_use_coupon_rate'] = user2_feature.buy_use_coupon.astype('float')/user2_feature.buy_total.astype('float')
user2_feature['user_coupon_transfer_rate'] = user2_feature.buy_use_coupon.astype('float')/user2_feature.buy_use_coupon.astype('float')
user2_feature.buy_total = user2_feature.buy_total.replace(np.nan,0)
user2_feature.coupon_received = user2_feature.coupon_received.replace(np.nan,0)
user2_feature.to_csv('data/ml100marathon-02-01/user2_feature.csv',index=None)
 
 
# 對數據集dataset1操作
user1 = feature1[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
t = user1[['user_id']]
# 去掉數據中重復的用戶ID
t.drop_duplicates(inplace=True)
# 用戶購買商品的種類
t1 = user1[pd.notnull(user1.date)][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)
# 使用了優惠券購買商品的用戶id和距離
t2 = user1[(pd.notnull(user1.date))&(pd.notnull(user1.coupon_id))][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.distance = t2.distance.astype('float')
t2.replace(-1,np.nan,inplace=True)
# 得到使用優惠券購買商品的用戶距商店的最短距離
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_dsitance'},inplace=True)
# 最大距離
t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)
# 平均距離
t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)
# 中位數距離
t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)
# 每個用戶使用優惠券購買的商品數量
t7 = user1[(pd.notnull(user1.date))&(pd.notnull(user1.coupon_id))][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()
# 購買商品的總數
t8 = user1[pd.notnull(user1.date)][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()
# 接收優惠券的總數
t9 = user1[pd.notnull(user1.coupon_id)][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()
# 收到優惠券的日期和使用之間的距離
t10 = user1[(pd.notnull(user1.date_received))&(pd.notnull(user1.date))][['user_id','date_received','date']]
t10['user_date_datereceived_gap'] = t10.date.to_string()+':'+t10.date_received.to_string()
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id','user_date_datereceived_gap']]
# 將用戶優惠券使用時間的間隔取平均值
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
# 間隔天數的最小值
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
# 間隔天數的最大值
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)
# 合併特徵
user1_feature = pd.merge(t,t1,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t3,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t4,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t5,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t6,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t7,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t8,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t9,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t11,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t12,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t13,on='user_id',how='left')
user1_feature.count_merchant = user1_feature.count_merchant.replace(np.nan,0)
user1_feature.buy_user_coupon = user1_feature.buy_use_coupon.replace(np.nan,0)
user1_feature['buy_use_coupon_rate'] = user1_feature.buy_use_coupon.astype('float')/user1_feature.buy_total.astype('float')
user1_feature['user_coupon_transfer_rate'] = user1_feature.buy_use_coupon.astype('float')/user1_feature.buy_use_coupon.astype('float')
user1_feature.buy_total = user1_feature.buy_total.replace(np.nan,0)
user1_feature.coupon_received = user1_feature.coupon_received.replace(np.nan,0)
user1_feature.to_csv('data/ml100marathon-02-01/user1_feature.csv',index=None)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:92: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:167: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#a

In [10]:
# 用戶和商店之間聯繫的特徵
# 對數據集feature3進行處理
 
# 用戶和商店之間聯繫的特徵
all_user_merchant = feature3[['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)
# 只保留銷售了商品的商戶id
t = feature3[['user_id','merchant_id','date']]
t = t[pd.notnull(t.date)][['user_id','merchant_id']]
# 用戶一共買了特定商戶多少商品
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)
t1 = feature3[['user_id','merchant_id','coupon_id']]
t1 = t1[pd.notnull(t1.coupon_id)][['user_id','merchant_id']]
# 用戶一共收到一個商戶的優惠劵數目
t['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace = True)
 
t2 = feature3[['user_id','merchant_id','date','date_received']]
t2 = t2[(pd.notnull(t2.date))&(pd.notnull(t2.date_received))][['user_id','merchant_id']]
# 用戶在一家商戶使用優惠券購買的商品數目
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace = True)
# 用戶在一家商家的說有記錄總數
t3 = feature3[['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)
# 用戶未使用優惠券購買的商品數目
t4 = feature3[['user_id','merchant_id','date','coupon_id']]
t4 = t4[(pd.notnull(t4.date))&(pd.isnull(t4.coupon_id))][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace = True)
user_merchant3 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t1,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t2,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t3,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t4,on=['user_id','merchant_id'],how='left')
# 都是針對一個商戶和一個用戶
user_merchant3['user_merchant_coupon_transfer_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_received.astype('float')
user_merchant3['user_merchant_coupon_buy_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float')/user_merchant3.user_merchant_buy_total.astype('float')
user_merchant3['user_merchant_rate'] = user_merchant3.user_merchant_buy_total.astype('float')/user_merchant3.user_merchant_any.astype('float')
user_merchant3['user_merchant_common_buy_rate'] = user_merchant3.user_merchant_buy_common.astype('float')/user_merchant3.user_merchant_buy_total.astype('float')
user_merchant3.to_csv('data/ml100marathon-02-01/user_merchant3.csv',index=None)
 
 
# 對於數據集feature2
all_user_merchant = feature2[['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)
t = feature2[['user_id','merchant_id','date']]
t = t[pd.notnull(t.date)][['user_id','merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)
t1 = feature2[['user_id','merchant_id','coupon_id']]
t1 = t1[pd.notnull(t1.coupon_id)][['user_id','merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace = True)
 
t2 = feature2[['user_id','merchant_id','date','date_received']]
t2 = t2[(pd.notnull(t2.date))&(pd.notnull(t2.date_received))][['user_id','merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace=True)
 
t3 = feature2[['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)
 
t4 = feature2[['user_id','merchant_id','date','coupon_id']]
t4 = t4[(pd.notnull(t4.date))&(pd.isnull(t4.coupon_id))][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace=True)
 
user_merchant2 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t1,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t2,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t3,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t4,on=['user_id','merchant_id'],how='left')
 
user_merchant2.user_merchant_buy_use_coupon = user_merchant2.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant2.user_merchant_buy_common = user_merchant2.user_merchant_buy_common.replace(np.nan,0)
 
user_merchant2['user_merchant_coupon_transfer_rate'] = user_merchant2.user_merchant_buy_use_coupon.astype('float')/user_merchant2.user_merchant_received.astype('float')
user_merchant2['user_merchant_coupon_buy_rate'] = user_merchant2.user_merchant_buy_use_coupon.astype('float')/user_merchant2.user_merchant_buy_total.astype('float')
user_merchant2['user_merchant_rate'] = user_merchant2.user_merchant_buy_total.astype('float')/user_merchant2.user_merchant_any.astype('float')
user_merchant2['user_merchant_common_buy_rate'] = user_merchant2.user_merchant_buy_common.astype('float')/user_merchant2.user_merchant_buy_total.astype('float')
user_merchant2.to_csv('data/ml100marathon-02-01/user_merchant2.csv',index=None)
 
 
# 對於數據集feature1
all_user_merchant = feature1[['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)
t = feature1[['user_id','merchant_id','date']]
t = t[pd.notnull(t.date)][['user_id','merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)
t1 = feature1[['user_id','merchant_id','coupon_id']]
t1 = t1[pd.notnull(t1.coupon_id)][['user_id','merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace = True)
 
t2 = feature1[['user_id','merchant_id','date','date_received']]
t2 = t2[(pd.notnull(t2.date))&(pd.notnull(t2.date_received))][['user_id','merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace=True)
 
t3 = feature1[['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)
 
t4 = feature1[['user_id','merchant_id','date','coupon_id']]
t4 = t4[(pd.notnull(t4.date))&(pd.isnull(t4.coupon_id))][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace=True)
 
user_merchant1 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t1,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t2,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t3,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t4,on=['user_id','merchant_id'],how='left')
 
user_merchant1.user_merchant_buy_use_coupon = user_merchant1.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant1.user_merchant_buy_common = user_merchant1.user_merchant_buy_common.replace(np.nan,0)
 
user_merchant1['user_merchant_coupon_transfer_rate'] = user_merchant1.user_merchant_buy_use_coupon.astype('float')/user_merchant1.user_merchant_received.astype('float')
user_merchant1['user_merchant_coupon_buy_rate'] = user_merchant1.user_merchant_buy_use_coupon.astype('float')/user_merchant1.user_merchant_buy_total.astype('float')
user_merchant1['user_merchant_rate'] = user_merchant1.user_merchant_buy_total.astype('float')/user_merchant1.user_merchant_any.astype('float')
user_merchant1['user_merchant_common_buy_rate'] = user_merchant1.user_merchant_buy_common.astype('float')/user_merchant1.user_merchant_buy_total.astype('float')
user_merchant1.to_csv('data/ml100marathon-02-01/user_merchant1.csv',index=None)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is try

In [28]:
# dataset1,2,3分別是劃分時間滑窗後的所提取的特徵的組合，方便接下來的劃分訓練集和測試集
# 此次合併後dataset1,2,3的特徵類型是一樣的，然後給dataset1,2添加標籤，標籤是get_label(s)這個函數所生成的
def get_label(row):
    if np.isnan(row['date_received']):
        return -1
    if not np.isnan(row['date']):
        td = pd.to_datetime(row['date'], format='%Y%m%d') -  pd.to_datetime(row['date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0   
 
coupon3 = pd.read_csv('data/ml100marathon-02-01/coupon3_feature.csv')
merchant3 = pd.read_csv('data/ml100marathon-02-01/merchant3_feature.csv')
user3 = pd.read_csv('data/ml100marathon-02-01/user3_feature.csv')
other_feature3 = pd.read_csv('data/ml100marathon-02-01/other_feature3.csv')
user_merchant3 = pd.read_csv('data/ml100marathon-02-01/user_merchant3.csv')
dataset3 = pd.merge(coupon3,merchant3,on='merchant_id',how='left')
dataset3 = pd.merge(dataset3,user3,on='user_id',how='left')
dataset3 = pd.merge(dataset3,user_merchant3,on=['user_id','merchant_id'],how='left')
dataset3 = pd.merge(dataset3,other_feature3,on=['user_id','coupon_id','date_received'],how='left')
dataset3.drop_duplicates(inplace=True)

 
dataset3.user_merchant_buy_total = dataset3.user_merchant_buy_total.replace(np.nan,0)
dataset3.user_merchant_any = dataset3.user_merchant_any.replace(np.nan,0)
dataset3.user_merchant_received = dataset3.user_merchant_received.replace(np.nan,0)
dataset3['is_weekend'] = dataset3.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
# get_dummies 进行one-hot编码
weekday_dummies = pd.get_dummies(dataset3.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset3 = pd.concat([dataset3,weekday_dummies],axis=1)
# dataset3.columns
dataset3.drop(['merchant_id','day_of_week','coupon_count'],axis=1,inplace=True)
dataset3 = dataset3.replace('null',np.nan)
dataset3.to_csv('data/ml100marathon-02-01/dataset3.csv',index=None)
 
coupon2 = pd.read_csv('data/ml100marathon-02-01/coupon2_feature.csv')
merchant2 = pd.read_csv('data/ml100marathon-02-01/merchant2_feature.csv')
user2 = pd.read_csv('data/ml100marathon-02-01/user2_feature.csv')
user_merchant2 = pd.read_csv('data/ml100marathon-02-01/user_merchant2.csv')
other_feature2 = pd.read_csv('data/ml100marathon-02-01/other_Feature2.csv')
dataset2 = pd.merge(coupon2,merchant2,on='merchant_id',how='left')
dataset2 = pd.merge(dataset2,user2,on='user_id',how='left')
dataset2 = pd.merge(dataset2,user_merchant2,on=['user_id','merchant_id'],how='left')
dataset2 = pd.merge(dataset2,other_feature2,on=['user_id','coupon_id','date_received'],how='left')
dataset2.drop_duplicates(inplace=True)
# dataset2.head()
 
dataset2.user_merchant_buy_total = dataset2.user_merchant_buy_total.replace(np.nan,0)
dataset2.user_merchant_any = dataset2.user_merchant_any.replace(np.nan,0)
dataset2.user_merchant_received = dataset2.user_merchant_received.replace(np.nan,0)
dataset2['is_weekend'] = dataset2.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset2.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset2 = pd.concat([dataset2,weekday_dummies],axis=1)
#dataset2['label'] = dataset2.date.astype('str') + ':' +  dataset2.date_received.astype('str')
dataset2['label'] = dataset2.apply(get_label, axis=1)
dataset2.drop(['merchant_id','day_of_week','date','date_received','coupon_id','coupon_count'],axis=1,inplace=True)
dataset2 = dataset2.replace('null',np.nan)
dataset2.to_csv('data/ml100marathon-02-01/dataset2.csv',index=None)
 
 
coupon1 = pd.read_csv('data/ml100marathon-02-01/coupon1_feature.csv')
merchant1 = pd.read_csv('data/ml100marathon-02-01/merchant1_feature.csv')
user1 = pd.read_csv('data/ml100marathon-02-01/user1_feature.csv')
user_merchant1 = pd.read_csv('data/ml100marathon-02-01/user_merchant1.csv')
other_feature1 = pd.read_csv('data/ml100marathon-02-01/other_feature1.csv')
dataset1 = pd.merge(coupon1,merchant1,on='merchant_id',how='left')
dataset1 = pd.merge(dataset1,user1,on='user_id',how='left')
dataset1 = pd.merge(dataset1,user_merchant1,on=['user_id','merchant_id'],how='left')
dataset1 = pd.merge(dataset1,other_feature1,on=['user_id','coupon_id','date_received'],how='left')
dataset1.drop_duplicates(inplace=True)
# print dataset1.shape
 
dataset1.user_merchant_buy_total = dataset1.user_merchant_buy_total.replace(np.nan,0)
dataset1.user_merchant_any = dataset1.user_merchant_any.replace(np.nan,0)
dataset1.user_merchant_received = dataset1.user_merchant_received.replace(np.nan,0)
dataset1['is_weekend'] = dataset1.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset1.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset1 = pd.concat([dataset1,weekday_dummies],axis=1)
#dataset1['label'] = dataset1.date.astype('str') + ':' +  dataset1.date_received.astype('str')
dataset1['label'] = dataset1.apply(get_label, axis=1)
dataset1.drop(['merchant_id','day_of_week','date','date_received','coupon_id','coupon_count'],axis=1,inplace=True)
dataset1 = dataset1.replace('null',np.nan)
dataset1.to_csv('data/ml100marathon-02-01/dataset1.csv',index=None)

In [29]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
 
dataset1 = pd.read_csv('data/ml100marathon-02-01/dataset1.csv')
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('data/ml100marathon-02-01/dataset2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('data/ml100marathon-02-01/dataset3.csv')
# 去重
dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)
 
dataset12 = pd.concat([dataset1,dataset2],axis=0)
# 再次組合成訓練集
dataset1_y = dataset1.label
dataset1_x = dataset1.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)  
# 'day_gap_before','day_gap_after' cause overfitting, 0.77
dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)
dataset3_preds = dataset3[['user_id','coupon_id','date_received']]
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after','weekday8'],axis=1)
# dataset3_x = dataset3.drop(['user_id','coupon_id','date_received'],axis=1)
 
# print(dataset1_x.shape,dataset2_x.shape,dataset3_x.shape)
 
dataset1 = xgb.DMatrix(dataset1_x,label=dataset1_y)
dataset2 = xgb.DMatrix(dataset2_x,label=dataset2_y)
dataset12= xgb.DMatrix(dataset12_x,label=dataset12_y)
dataset3 = xgb.DMatrix(dataset3_x)
# 在XGBoost中，要將處理的數據存儲在對象DMatrix中，方便下一步處理

In [30]:
params={'booster':'gbtree',
        'objective': 'rank:pairwise',
        'eval_metric':'auc',
        'gamma':0.1,
        'min_child_weight':1.1,
        'max_depth':5,
        'lambda':10,
        'subsample':0.7,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.01,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }
watchlist = [(dataset12,'train')]

# 模型訓練
model = xgb.train(params,dataset12,num_boost_round=3500,evals=watchlist)
 


[23:19:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-auc:0.821032
[23:19:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-auc:0.833978
[23:19:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-auc:0.838165
[23:19:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[3]	train-auc:0.844852
[23:19:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-auc:0.85016
[23:19:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[5]	train-auc:0.851194
[23:19:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[6]	train-auc:0.851044
[23:19:48] src/tree/updater_prune.cc:74: tree pruning en

[23:19:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[61]	train-auc:0.856141
[23:19:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[62]	train-auc:0.85622
[23:19:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[63]	train-auc:0.85628
[23:19:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[64]	train-auc:0.856227
[23:19:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[65]	train-auc:0.856629
[23:19:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[66]	train-auc:0.856818
[23:19:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[67]	train-auc:0.856759
[23:19:56] src/tree/updater_prune.cc:74: tree prun

[23:20:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[122]	train-auc:0.861071
[23:20:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[123]	train-auc:0.861117
[23:20:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[124]	train-auc:0.861109
[23:20:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[125]	train-auc:0.861104
[23:20:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[126]	train-auc:0.861172
[23:20:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[127]	train-auc:0.861224
[23:20:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[128]	train-auc:0.861238
[23:20:05] src/tree/updater_prune.cc:74: 

[23:20:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[182]	train-auc:0.863481
[23:20:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[183]	train-auc:0.863556
[23:20:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[184]	train-auc:0.863588
[23:20:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[185]	train-auc:0.863588
[23:20:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[186]	train-auc:0.863598
[23:20:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[187]	train-auc:0.863655
[23:20:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[188]	train-auc:0.863655
[23:20:14] src/tree/updater_prune.cc:74: 

[23:20:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[242]	train-auc:0.865359
[23:20:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[243]	train-auc:0.865377
[23:20:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[244]	train-auc:0.865374
[23:20:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[245]	train-auc:0.865436
[23:20:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[246]	train-auc:0.865467
[23:20:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[247]	train-auc:0.865566
[23:20:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[248]	train-auc:0.865613
[23:20:22] src/tree/updater_prune.cc:74: 

...

[23:28:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[3242]	train-auc:0.89167
[23:28:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3243]	train-auc:0.891673
[23:28:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[3244]	train-auc:0.891677
[23:28:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[3245]	train-auc:0.89168
[23:28:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3246]	train-auc:0.891684
[23:28:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3247]	train-auc:0.891688
[23:28:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3248]	train-auc:0.89169
[23:28:21] src/tree/updater_prune.cc:

[23:28:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[3302]	train-auc:0.891854
[23:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3303]	train-auc:0.891857
[23:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3304]	train-auc:0.891861
[23:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[3305]	train-auc:0.891864
[23:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3306]	train-auc:0.891866
[23:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3307]	train-auc:0.89187
[23:28:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3308]	train-auc:0.891873
[23:28:31] src/tree/updater_prune.c

[23:28:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3362]	train-auc:0.892042
[23:28:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3363]	train-auc:0.892045
[23:28:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2 pruned nodes, max_depth=5
[3364]	train-auc:0.892048
[23:28:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3365]	train-auc:0.892054
[23:28:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3366]	train-auc:0.892058
[23:28:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[3367]	train-auc:0.892061
[23:28:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[3368]	train-auc:0.892064
[23:28:40] src/tree/updater_prune.

[23:28:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 2 pruned nodes, max_depth=5
[3422]	train-auc:0.892222
[23:28:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3423]	train-auc:0.892224
[23:28:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[3424]	train-auc:0.892226
[23:28:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3425]	train-auc:0.892229
[23:28:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[3426]	train-auc:0.892232
[23:28:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[3427]	train-auc:0.892237
[23:28:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3428]	train-auc:0.89224
[23:28:50] src/tree/updater_prune.c

[3481]	train-auc:0.892387
[23:28:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[3482]	train-auc:0.892389
[23:28:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[3483]	train-auc:0.892391
[23:28:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[3484]	train-auc:0.892395
[23:28:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[3485]	train-auc:0.892399
[23:28:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 2 pruned nodes, max_depth=5
[3486]	train-auc:0.892403
[23:28:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[3487]	train-auc:0.892405
[23:28:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[3488]	train-auc:0.892407
[23:29:0

In [31]:
# 對dataset3進行預測
dataset3_preds['label'] = model.predict(dataset3)
dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds.label.values.reshape(-1, 1))

dataset3_preds = dataset3_preds[~dataset3_preds.coupon_id.isna()]
dataset3_preds.reset_index(drop=True, inplace=True)

dataset3_preds.head()


,user_id,coupon_id,date_received,label
0,1439408,11002.0,20160528.0,0.353321
1,1439408,8591.0,20160613.0,0.564775
2,1439408,8591.0,20160516.0,0.614043
3,2029232,1532.0,20160530.0,0.385282
4,2029232,12737.0,20160519.0,0.503747


In [35]:
print(dataset3_preds.shape)

(304096, 5)


In [32]:
def convertingToString(row):
    if (pd.isnull(row)) or (row==-1):
        return ''
    else:
        return str(int(row))

dataset3_preds.loc[:, "user_id"] = dataset3_preds["user_id"].apply(convertingToString)
dataset3_preds.loc[:, "coupon_id"] = dataset3_preds["coupon_id"].apply(convertingToString)
dataset3_preds.loc[:, "date_received"] = dataset3_preds["date_received"].apply(convertingToString)
dataset3_preds["uid"] = dataset3_preds[["user_id", "coupon_id", "date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
dataset3_preds.reset_index(drop=True, inplace=True)

In [33]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = dataset3_preds.groupby("uid", as_index=False).mean()
out = out[["uid", "label"]]
out.columns = ["uid", "label"]
out.to_csv("ml100marathon-midterm-exam-ver5.csv", header=["uid", "label"], index=False) # submission format
out.head()

,uid,label
0,1000020_2705_20160519,0.474562
1,1000020_8192_20160513,0.452555
2,1000065_1455_20160527,0.416022
3,1000085_8067_20160513,0.374540
4,1000086_2418_20160613,0.440521


In [34]:
# feature_score來保存特徵對標籤的影響因子
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
     
with open('xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)